In [1]:
!pip install annoy
!pip install implicit

In [2]:
import annoy
import numpy as np
import pandas as pd
from surprise import Dataset
from surprise import Reader
from surprise import SVD, evaluate, accuracy
from surprise import dump
from surprise.model_selection import cross_validate, train_test_split
from surprise.model_selection import GridSearchCV 

In [20]:
from surprise import NMF
from surprise import KNNBasic
from surprise import KNNWithMeans
from surprise import KNNBaseline
from surprise import KNNWithZScore

In [28]:
from tqdm import tqdm_notebook as tqdm

In [4]:
df = pd.read_csv('item_item_test.csv')
df.head(30)

,Unnamed: 0,userId,movieId,rating
0,90,4,181,0.5
1,296,4,1760,0.5
2,484,4,3593,1.0
3,1631,19,181,2.0
4,2655,38,4125,1.0
5,4450,54,810,3.0
6,5189,56,4255,3.5
7,6455,72,90890,3.5
8,6506,72,125916,0.5
9,7119,75,125916,2.5


In [5]:
df.drop('Unnamed: 0', inplace=True,axis=1)

In [6]:
df.head()

,userId,movieId,rating
0,4,181,0.5
1,4,1760,0.5
2,4,3593,1.0
3,19,181,2.0
4,38,4125,1.0


In [15]:
def rec_gridsearch(model,param_grid,cv):
    gs = GridSearchCV(model, param_grid, measures=['rmse', 'mae'], cv=cv)
    gs.fit(data)
    # best RMSE score
    print(gs.best_score['rmse'])

    # combination of parameters that gave the best RMSE score
    print(gs.best_params['rmse'])

## SVD

In [14]:
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(df,reader)

In [17]:
svd = SVD()
cross_validate(svd, data, measures=['RMSE', 'MAE'], cv=10, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 10 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Fold 6  Fold 7  Fold 8  Fold 9  Fold 10 Mean    Std     
RMSE (testset)    1.0535  1.0567  1.0555  1.0647  1.0666  1.0539  1.0719  1.0734  1.0757  1.0692  1.0641  0.0081  
MAE (testset)     0.8516  0.8560  0.8588  0.8659  0.8597  0.8581  0.8628  0.8630  0.8739  0.8638  0.8614  0.0058  
Fit time          3.92    3.90    4.00    4.64    4.04    3.98    3.97    4.68    4.43    4.69    4.23    0.32    
Test time         0.05    0.05    0.14    0.05    0.05    0.05    0.06    0.05    0.05    0.05    0.06    0.03    


{'test_rmse': array([1.05349738, 1.05670008, 1.05550427, 1.06472972, 1.0665943 ,
        1.05393943, 1.0718726 , 1.07340582, 1.07567147, 1.06921951]),
 'test_mae': array([0.85159141, 0.85604305, 0.85884696, 0.86593192, 0.85970814,
        0.85806593, 0.86278583, 0.86295761, 0.87386728, 0.86381945]),
 'fit_time': (3.9230339527130127,
  3.903351068496704,
  3.995553970336914,
  4.640155076980591,
  4.039940118789673,
  3.981184720993042,
  3.9710001945495605,
  4.6831419467926025,
  4.434258937835693,
  4.688935995101929),
 'test_time': (0.05298614501953125,
  0.053289175033569336,
  0.137437105178833,
  0.05352187156677246,
  0.054746150970458984,
  0.05100202560424805,
  0.056944847106933594,
  0.05170726776123047,
  0.051301002502441406,
  0.05142498016357422)}

In [25]:
param_grid = {'n_epochs': [10, 20, 50], 'n_factors': [1,5,10],'lr_all': [0.001, 0.005, 0.01,0.05],
              'reg_all': [0.4, 0.6], 'biased': [True, False],}
rec_gridsearch(SVD,param_grid,6)

1.0372939179830616
{'n_epochs': 50, 'n_factors': 10, 'lr_all': 0.01, 'reg_all': 0.4, 'biased': True}


## NMF

In [26]:
nmf = NMF()
cross_validate(nmf, data, measures=['RMSE', 'MAE'], cv=10, verbose=True)

Evaluating RMSE, MAE of algorithm NMF on 10 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Fold 6  Fold 7  Fold 8  Fold 9  Fold 10 Mean    Std     
RMSE (testset)    1.0579  1.0626  1.0713  1.0747  1.0507  1.0627  1.0598  1.0679  1.0507  1.0569  1.0615  0.0076  
MAE (testset)     0.7927  0.7962  0.8054  0.8051  0.7865  0.7903  0.7941  0.8035  0.7853  0.7860  0.7945  0.0075  
Fit time          8.39    8.93    9.41    10.92   9.63    9.71    11.70   12.68   10.33   8.70    10.04   1.30    
Test time         0.07    0.13    0.07    0.08    0.25    0.23    0.08    0.11    0.06    0.06    0.11    0.07    


{'test_rmse': array([1.05787133, 1.06261884, 1.0712683 , 1.07469762, 1.05066367,
        1.06269279, 1.05977168, 1.06787728, 1.05067864, 1.05689128]),
 'test_mae': array([0.79268041, 0.79622685, 0.8053805 , 0.80510888, 0.78648825,
        0.79025716, 0.79406643, 0.80348372, 0.78527914, 0.78599573]),
 'fit_time': (8.385046005249023,
  8.929218769073486,
  9.407428979873657,
  10.915997982025146,
  9.629431962966919,
  9.705810070037842,
  11.69683575630188,
  12.680475234985352,
  10.332500219345093,
  8.695887088775635),
 'test_time': (0.0735630989074707,
  0.13040709495544434,
  0.0650639533996582,
  0.07552385330200195,
  0.24634122848510742,
  0.23092412948608398,
  0.08028912544250488,
  0.11409163475036621,
  0.057929039001464844,
  0.059168100357055664)}

In [29]:
# Not tuning LR/Reg until biased is determined to be true or not. 
param_grid = {'n_epochs': [10, 20, 50], 'n_factors': [1,5,10], 'biased': [True, False],}
tqdm(rec_gridsearch(NMF,param_grid,9))

1.058634612169605
{'n_epochs': 50, 'n_factors': 1, 'biased': False}


0it [00:00, ?it/s]